In [8]:
import pandas as pd
import os
from pathlib import Path
from datetime import datetime, timedelta
import json
from copy import deepcopy

In [9]:
folder = Path('eoles') / Path('inputs') / Path('xps')

date = datetime.now()
date = date.strftime("%Y%m%d")  # formatting
folder = folder / Path(str(date))

# add one day to date object
# date = date + timedelta(days=1)
# check if folder is a folder with library pathlib
folder.mkdir(parents=True, exist_ok=True)

base = {
    "no_subsidies": True,
    "subsidies_specified": False,
    "calibration": True,
    "max_iter": 100,
    "health": True,
    "discount_rate": 0.032,
    "weather_year": 2006,
    "carbon_constraint": True,
    "fix_sub_heater": False,
    "fix_sub_insulation": False,
    "list_year": [2020, 2025, 2030, 2035, 2040, 2045],
    "list_trajectory_scc": [0, 250, 350, 500, 650, 775],
    "eoles": {
        "biomass_potential_scenario": "S3",
        "aggregated_potential": True,
        "maximum_capacity_scenario": "N1"
    },
    "subsidy": {
            "proportional_uniform": None,
            "heater": {
                "policy": "subsidy_ad_valorem",
                "proportional": None,
                "cap": None
            },
            "insulation": {
                "policy": "subsidy_ad_valorem",
                "rational_behavior": False,
                "target": None,
                "proportional": None,
                "cap": None
            }
    },
    "scenario_cost_eoles": {
        "fix_capacities": {
            "uiom": 0,
            "CTES": 0
        }
    }
}

folder_base = folder / Path('base.json')
with open(folder_base, "w") as outfile:
    outfile.write(json.dumps(base, indent=4))


'eoles/inputs/xps/20240221/settings_framework.json'

In [21]:
scenarios = {
    'biogas': ['reference', 'Biogas-'],
    'capacity': ['reference', 'Nuc-', 'Ren-'],
    # 'demand': ['reference', 'Sufficiency', 'Reindustrialisation'],
    'policy': ['Ban', 'BanNoPolicy', 'Package2024', 'NoPolicy'],
    'learning': ['reference', 'Learning+'],
    # 'profile': ['reference', 'ProfileFlat'],
    'elasticity': ['reference', 'Elasticity-']
}

map_values = {
    'Biogas-': 'S2',
    'Nuc-': 'N1nuc',
    'Ren-': 'N1ren2',
    'Sufficiency': 'Sobriete',
    'Reindustrialisation': 'Reindustrialisation',
    'Ban': {
        "file": "project/input/policies/policies_2024.json",
        "restriction_gas": {
            "start": 2025,
            "end": 2051,
            "value": "Natural gas",
            "policy": "restriction_energy"
    }},
    'BanNoPolicy':  {
            "file": "project/input/policies/policies_calibration.json",
            "restriction_gas": {
                "start": 2025,
                "end": 2051,
                "value": "Natural gas",
                "policy": "restriction_energy"
    }},
    'Package2024': {
            "file": "project/input/policies/policies_2024.json"
    },
    'NoPolicy': {
            "file": "project/input/policies/policies_calibration.json"
    },
    'Learning+': {
          "technical_progress": {
            "heater": {
              "activated": True,
              "start": 2019,
              "end": 2035,
              "value_end": -0.5
            }
          }
        },
    'ProfileFlat': {
            "hourly_profile": "project/input/technical/hourly_profile_flat.csv"
        },
    'Elasticity-': {
          "scale": {
            "option": "price_elasticity",
            "target": -0.5
          }}
}

mapping_scenarios_to_configs = {
    'biogas': ['supply', 'biomass_potential_scenario'],
    'capacity': ['supply', 'maximum_capacity_scenario'],
    'policy': ['policies'],
    'demand': ['supply', 'demand_scenario'],
    'technical': ['demand'],
    'learning': ['demand', 'technical'],
    'profile': ['demand', 'technical'],
    'elasticity': ['demand', 'switch_heater']
}



In [10]:
keys, values = zip(*scenarios.items())

In [11]:
import itertools
permutations_names = [dict(zip(keys, v)) for v in itertools.product(*values)]
permutations_names = {'S{}'.format(n): v for n, v in enumerate(permutations_names)}

In [20]:
config_additional = {
    "name_config": "reference",
    "policies": {
        "file": "project/input/policies/current/policies_ambitious.json"
    },
    "greenfield": False,
    "prices_constant": False,
    "price_feedback": True,
    "biomass_potential_scenario": "S3",
    "aggregated_potential": True,
    "maximum_capacity_scenario": "N1",
    "lifetime_insulation": 5,
    "subsidies_heater": None,
    "subsidies_insulation": None,
    "optim_eoles": True,
    "carbon_budget": "carbon_budget_tight",
    "district_heating_potential": None,
    "dict_configs": None,
    "cap_MWh": 1000,
    "cap_tCO2": 2000,
    "demand_scenario": "Reference",
    "method_health_cost": "epc"
}

for key, values in permutations_names.items():  #key: 'S0', values= 'biogas'
    new_config = config_additional.copy()
    new_config['name_config'] = key
    for i,v in values.items():  # i: 'biogas', v: 'Biogas-'
        if v == 'reference':  # no modification to the configuration
            pass
        else:
            if mapping_scenarios_to_configs[i][0] == 'supply':
                new_config[mapping_scenarios_to_configs[i][1]] = deepcopy(map_values[v])
                # print(new_config)
            elif mapping_scenarios_to_configs[i][0] == 'demand':
                if mapping_scenarios_to_configs[i][1] in new_config.keys():
                    new_config[mapping_scenarios_to_configs[i][1]].update(deepcopy(map_values[v]))
                else:
                    new_config[mapping_scenarios_to_configs[i][1]] = deepcopy(map_values[v])
            elif mapping_scenarios_to_configs[i][0] == 'policies':
                new_config['policies'] = deepcopy(map_values[v])
            else:
                raise KeyError('Key not found')
    folder_additional = folder / Path(key + '.json')
    with open(folder_additional, "w") as outfile:
        outfile.write(json.dumps(new_config, indent=4))

pd.DataFrame.from_dict(permutations_names, orient='index').to_csv(folder / Path('scenarios.csv'))

False